<a href="https://colab.research.google.com/github/GuyWhoCodesThings/Custom-Functions-for-Scratch-Neural-Network/blob/main/Max_Pooling_Layer_for_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Max Pooling layer with some other helper methods to make it work

In [ ]:
import numpy as np

In [1]:
class Layer:
  def __init__(self):
    self.input = None
    self.output = None
  def forward(self, input):
    pass
  def backward(self, output_gradient, alpha):
    pass

In [2]:
def get_ordered_blocks(input, depth, block_size, input_shape):
    ordered_blocks = input.reshape(depth, -1, block_size, block_size)
    reshaped_arr = np.zeros(input_shape)
    for i in range(depth):
      split_blocks = np.array(np.hsplit(ordered_blocks[i], block_size))
      #splits array of blocks evenly with respect to the original image length and block size
      split_blocks = np.array(np.hsplit(split_blocks, 12))
      #reshape array to get it into a sqaure shape like orignal image matrix
      reshaped_arr[i] = split_blocks.flatten().reshape(1, input_shape[2], input_shape[1])
    return reshaped_arr

In [3]:
def expand_output(output_gradient, depth, height, width, block_size, output_height, output_width):
  output_gradient = output_gradient.reshape(depth, output_height, output_width)
  input_grad = np.zeros((depth, int(output_height*output_height), block_size**2))
  for i in range(depth):
    input_grad[i] = np.repeat(output_gradient[i], block_size**2).reshape(-1,block_size**2)
  return get_ordered_blocks(input_grad, depth, block_size, (depth, height, width))


In [4]:
class MaxPool2d(Layer):
  def __init__(self, input_shape, block_size):
    self.block_size = block_size
    self.input_shape = input_shape
    input_depth, input_height, input_width = input_shape
    self.depth = input_depth
    self.input_height = input_height
    self.input_width = input_width
    self.output_width = int(self.input_width / self.block_size)
    self.output_height = int(self.input_height / self.block_size)
    
  def forward(self, input):
    self.block_image = view_as_blocks(input, (1,self.block_size, self.block_size)).reshape(self.depth,-1, self.block_size**2)
    self.output = np.amax(self.block_image, axis=2).reshape(-1, self.output_height, self.output_width)
    return self.output
    
  def backward(self, output_gradient, learning_rate):
    max_indices = np.argmax(self.block_image, axis=2)
    max_indices = max_indices.reshape(self.depth, -1, 1)
    input_gradient = np.zeros_like(self.block_image).reshape(self.depth, -1, self.block_size**2)
    np.put_along_axis(input_gradient, max_indices, 1, axis=2)
    return np.multiply(get_ordered_blocks(input_gradient, self.depth, self.block_size, self.input_shape),
                      expand_output(output_gradient, self.depth, self.input_height, self.input_width,
                      self.block_size, self.output_width, self.output_height))